<a target="_blank" href="https://colab.research.google.com/github/tuankhoin/MCEN90048-EEG-Grasp-and-Lift-Detection/blob/main/code.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [1]:
import os
import json
import zipfile
from pathlib import Path

try:
  # See if you are running on colab
  from google.colab import drive 

  # Download the dataset
  api_token = {"username":"tuankhoinguyen","key":"6b992b229c6a4d8f6a8b30d1c101507d"}
  kaggle_path = Path('/root/.kaggle')
  os.makedirs(kaggle_path, exist_ok=True)

  with open (kaggle_path/'kaggle.json', 'w') as handl:
      json.dump(api_token,handl)
  os.chmod(kaggle_path/'kaggle.json', 600)  
  !kaggle competitions download -c grasp-and-lift-eeg-detection

  # Unzipping datasets
  for z in ['grasp-and-lift-eeg-detection','train']:
    print(f'Unzipping {z}.zip')
    with zipfile.ZipFile(f'{z}.zip', 'r') as zip_ref:
      zip_ref.extractall('')
  print('All good!')
  # Removing the zip files to save up VM space
  !rm *.zip
except:
  pass

100% 1.02G/1.02G [00:36<00:00, 32.6MB/s]
100% 1.02G/1.02G [00:36<00:00, 29.8MB/s]
Unzipping grasp-and-lift-eeg-detection.zip
Unzipping train.zip
All good!


In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

import glob
import tqdm
import pywt
import warnings
warnings.filterwarnings('ignore')

In [3]:
datas = glob.glob('/content/train/*_data.csv')
print(datas)

['/content/train/subj5_series6_data.csv', '/content/train/subj3_series7_data.csv', '/content/train/subj11_series5_data.csv', '/content/train/subj3_series3_data.csv', '/content/train/subj2_series2_data.csv', '/content/train/subj8_series7_data.csv', '/content/train/subj2_series6_data.csv', '/content/train/subj12_series7_data.csv', '/content/train/subj9_series2_data.csv', '/content/train/subj4_series4_data.csv', '/content/train/subj7_series8_data.csv', '/content/train/subj2_series4_data.csv', '/content/train/subj9_series3_data.csv', '/content/train/subj3_series5_data.csv', '/content/train/subj7_series2_data.csv', '/content/train/subj4_series5_data.csv', '/content/train/subj3_series2_data.csv', '/content/train/subj1_series5_data.csv', '/content/train/subj12_series3_data.csv', '/content/train/subj1_series6_data.csv', '/content/train/subj7_series5_data.csv', '/content/train/subj10_series3_data.csv', '/content/train/subj11_series4_data.csv', '/content/train/subj6_series4_data.csv', '/content/

In [4]:
dir = 'train_filtered'
T=0.8
w='db2'

if not os.path.exists(dir):
  os.mkdir(dir)

for d in tqdm.tqdm(datas):
  fname = d.split('/')[-1][:-8]
  df = pd.read_csv(d).iloc[:,1:]
  out = pd.DataFrame()
  for c in df.columns:
    coeffs = pywt.wavedec(df[c],w)
    new_c = [pywt.threshold(co, T*max(co)) if i else co for i,co in enumerate(coeffs)]
    out[c] = pywt.waverec(new_c,w).astype('float16')
  out.to_csv(f'{dir}/{fname}_filtered.csv')

100%|██████████| 96/96 [07:18<00:00,  4.57s/it]


In [5]:
!zip -r /content/filtered.zip /content/train_filtered

  adding: content/train_filtered/ (stored 0%)
  adding: content/train_filtered/subj1_series6__filtered.csv (deflated 89%)
  adding: content/train_filtered/subj1_series7__filtered.csv (deflated 86%)
  adding: content/train_filtered/subj1_series1__filtered.csv (deflated 87%)
  adding: content/train_filtered/subj8_series2__filtered.csv (deflated 87%)
  adding: content/train_filtered/subj10_series8__filtered.csv (deflated 84%)
  adding: content/train_filtered/subj4_series4__filtered.csv (deflated 89%)
  adding: content/train_filtered/subj7_series2__filtered.csv (deflated 87%)
  adding: content/train_filtered/subj2_series7__filtered.csv (deflated 86%)
  adding: content/train_filtered/subj6_series6__filtered.csv (deflated 90%)
  adding: content/train_filtered/subj1_series2__filtered.csv (deflated 89%)
  adding: content/train_filtered/subj4_series2__filtered.csv (deflated 89%)
  adding: content/train_filtered/subj7_series7__filtered.csv (deflated 83%)
  adding: content/train_filtered/subj6_se

In [9]:
# print(datas)
# print(out.dtypes)
# filt_datas = glob.glob('/content/train/*_data.csv')
for d in tqdm.tqdm(datas):
  print(d)
  fname = d.split('/')[-1][:-8]
  print(fname)
  df = pd.read_csv(d).iloc[:,1:]
  break


  0%|          | 0/96 [00:00<?, ?it/s]

/content/train/subj4_series5_data.csv
subj4_series5_


  0%|          | 0/96 [00:00<?, ?it/s]


In [12]:
# df.info
# df.head()
print(len(datas))

96


In [1]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, ZeroPadding2D, Conv2D, BatchNormalization, Activation, MaxPool2D, AveragePooling2D, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers.schedules import PiecewiseConstantDecay
import pandas as pd
from collections import Counter
from tensorboard.plugins import projector
import imageio


In [5]:
# Define identity block function
def identity_block(x, filter):
    # the input tensor separated as residue 
    x_skip = x
    # Layer 1
    x = tf.keras.layers.Conv2D(filter, (3,1), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)    #NHWC
    x = tf.keras.layers.Activation('relu')(x)
    # Layer 2
    x = tf.keras.layers.Conv2D(filter, (3,1), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)    #NHWC
    # Add Residue
    x = tf.keras.layers.Add()([x, x_skip])     
    x = tf.keras.layers.Activation('relu')(x)
    return x

# Define convolution block function
def convolutional_block(x, filter):
    # the input tensor separated as residue 
    x_skip = x
    # Layer 1
    x = tf.keras.layers.Conv2D(filter, (3,1), padding = 'same', strides = (2,1))(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation('relu')(x)
    # Layer 2
    x = tf.keras.layers.Conv2D(filter, (3,1), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    # Processing Residue with conv(1,1)
    x_skip = tf.keras.layers.Conv2D(filter, (1,1), strides = (2,1))(x_skip)
    # Add Residue
    x = tf.keras.layers.Add()([x, x_skip])     
    x = tf.keras.layers.Activation('relu')(x)
    return x

shape = (32, 1, 1) # CHANGE ACCORDING TO DATA
classes = 6         

# Setup Input Layer
x_input = tf.keras.layers.Input(shape)
x = tf.keras.layers.ZeroPadding2D((3, 0))(x_input)    #  to ensure that the dimensions of the output are the same as the input after convolution operations
x = tf.keras.layers.Conv2D(64, kernel_size=(7,1), strides=2, padding='same')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same')(x)

# Define size of sub-blocks and initial filter size
block_layers = [3, 4, 6, 3]
filter_size = 64

# Add the Resnet Blocks
for i in range(len(block_layers)):
    if i == 0:
        # For block 1 Convolutional block not needed
        for j in range(block_layers[i]):
            x = identity_block(x, filter_size)
    else:
        # For all the other blocks
        # The filter size will go on increasing by a factor of 2
        filter_size = filter_size*2
        # One Convolutional Block followed by Identity blocks
        x = convolutional_block(x, filter_size)
        for j in range(block_layers[i] - 1):
            x = identity_block(x, filter_size)

# End Dense Network
x = tf.keras.layers.AveragePooling2D((2,1), padding = 'same')(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(512, activation = 'relu')(x)
x = tf.keras.layers.Dense(classes, activation='sigmoid')(x)

model = tf.keras.models.Model(inputs = x_input, outputs = x, name = "Residual CNN")
model.summary()


Model: "Residual CNN"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 32, 1, 1)]   0           []                               
                                                                                                  
 zero_padding2d_3 (ZeroPadding2  (None, 38, 1, 1)    0           ['input_4[0][0]']                
 D)                                                                                               
                                                                                                  
 conv2d_108 (Conv2D)            (None, 19, 1, 64)    512         ['zero_padding2d_3[0][0]']       
                                                                                                  
 batch_normalization_99 (BatchN  (None, 19, 1, 64)   256         ['conv2d_108[0][0]']  

In [4]:
# batch_size = 128        # CHANGE AS REQUIRED
# num_instance = 50000    # CHANGE AS REQUIRED
# # Let's say, if we want to decay the learning rate at 20, 40, 80 epochs
# boundary = (20*num_instance // batch_size, 40*num_instance // batch_size, 80*num_instance // batch_size)
# value = (1e-3, 5e-4, 1e-4, 5e-5)
# learning_rate = PiecewiseConstantDecay(boundary, value)
# print(learning_rate)

# # in the loss, from_logits=True because we use linear activation at the last layer
# # we also try label smoothing
# model.compile(
#    optimizer=tf.keras.optimizers.Adam(learning_rate),
#     loss=tf.keras.losses.binary_crossentropy, 
#     metrics=['accuracy'])
# #loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True, label_smoothing=0.1),

# print('Model compiled.')

Model compiled.


In [ ]:
# # Train the model
# epochs = 50           # CHANGE AS REQUIRED
# history = model.fit(x_tr, y_tr, epochs=epochs, batch_size=batch_size)

In [ ]:
# # To evaluate the model with labels:
# model.evaluate(x_te, y_te, batch_size=100)